In [1]:
%pip install tensorflow==2.15.1 keras==2.15.0 pillow numpy  

In [1]:
!python --version
!python3.10 - <<'PY'
import sys, tensorflow as tf, keras, PIL
print("Python:", sys.version)
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("Pillow:", PIL.__version__)

Python 3.10.11


<< was unexpected at this time.



Python: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
TensorFlow: 2.15.1
Keras: 2.15.0
Pillow: 11.3.0


In [ ]:
import os
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import shutil

# ==== Path ====
IMAGE_FOLDER = r"YOUR_IMAGE_FOLDER"  # Replace with your image folder path
MODEL_PATH = r"model.h5"  # Replace with your model path
LABEL_PATH = r"labels.txt" # Replace with your labels file path

# ==== Create folder ====
BILL_FOLDER = os.path.join(IMAGE_FOLDER, "bill")
NO_BILL_FOLDER = os.path.join(IMAGE_FOLDER, "no_bill")

os.makedirs(BILL_FOLDER, exist_ok=True)
os.makedirs(NO_BILL_FOLDER, exist_ok=True)

# ==== set model ====
np.set_printoptions(suppress=True)
model = load_model(MODEL_PATH, compile=False)
class_names = open(LABEL_PATH, "r").readlines()

print("labels and model loaded successfully")

# ==== scan images ====
for filename in os.listdir(IMAGE_FOLDER):
    if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    image_path = os.path.join(IMAGE_FOLDER, filename)
    print(f"\n--- Processing: {filename} ---")

    try:
        # Load image
        image = Image.open(image_path).convert("RGB")

        # Resize & normalize
        size = (224, 224)
        image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
        image_array = np.asarray(image)
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Predict
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        data[0] = normalized_image_array
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index].strip()
        confidence_score = prediction[0][index]

        print(f"→ Class: {class_name} | Confidence: {confidence_score:.4f}")

        # ==== Processingg ====
        if class_name == "1 Bill":
            shutil.copy(image_path, os.path.join(BILL_FOLDER, filename))
            print("saved to folder: bill")
        elif class_name == "0 Not Bill":
            shutil.copy(image_path, os.path.join(NO_BILL_FOLDER, filename))
            print("saved to folder: no_bill")

    except Exception as e:
        print(f"error in {filename}: {e}")

print("\n============================")
print("Classification completed.")
print(f"bill images: {BILL_FOLDER}")
print(f"No bill images: {NO_BILL_FOLDER}")
print("============================")


Model & labels loaded successfully!

--- Processing: DLDR5BP27N2_15.jpg ---
1/1 [==============================] - 2s 2s/step
→ Class: 1 Bill | Confidence: 0.9997
📁 Đã lưu vào thư mục: bill

--- Processing: DLPA5BP22BK_10.jpg ---
1/1 [==============================] - 0s 61ms/step
→ Class: 1 Bill | Confidence: 0.9998
📁 Đã lưu vào thư mục: bill

--- Processing: DLPA5BP22BK_11.jpg ---
1/1 [==============================] - 0s 56ms/step
→ Class: 1 Bill | Confidence: 1.0000
📁 Đã lưu vào thư mục: bill

--- Processing: DLPA5BP22BK_12.jpg ---
1/1 [==============================] - 0s 58ms/step
→ Class: 0 Not Bill | Confidence: 0.9997
📁 Đã lưu vào thư mục: no_bill

--- Processing: DLSH5BI42N2_20.jpg ---
1/1 [==============================] - 0s 60ms/step
→ Class: 0 Not Bill | Confidence: 0.9997
📁 Đã lưu vào thư mục: no_bill

--- Processing: HSDR5B402T2_00.png ---
1/1 [==============================] - 0s 61ms/step
→ Class: 0 Not Bill | Confidence: 1.0000
📁 Đã lưu vào thư mục: no_bill

--- Proc